# Preliminaries

In [2]:
!pip install gpytorch
  
import math
import torch
import gpytorch 
import sklearn
from sklearn.feature_selection import mutual_info_regression
from matplotlib import pyplot as plt
from functools import reduce

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


%matplotlib inline
%load_ext autoreload
%autoreload 2



     |████████████████████████████████| 215kB 4.8MB/s 
  Created wheel for gpytorch: filename=gpytorch-0.3.5-py2.py3-none-any.whl size=349719 sha256=e40949f394ea8f8e45b4a461c9f35336ee1ab797873cbc713d027aa9d8aee1ca
  Stored in directory: /root/.cache/pip/wheels/d6/31/88/c43a94e0073a54056ac663366f2195de36535b38a81a378196
Successfully built gpytorch


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/MSC Thesis

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/MSC Thesis


In [0]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

predictors = pd.read_csv('pred_inputs.csv')
predict_these = np.unique(predictors['INSTRUMENT'])

def from_csv(instrument):
    df = pd.read_csv('Data/' + instrument + '.csv')
    df['date'] = pd.to_datetime(df['date']).dt.date
    df.set_index('date',inplace=True)
    df = df.sort_values(by = 'date')
    df.columns = [instrument]
    
    return df
  
def preprocessing(commodity):
    #####################################################################################
    # This ensures chronological order (some time series are NOT in order)              #
    #####################################################################################
    
    prices_df = db.get_instrument_data(commodity)
    prices_df = prices_df.sort_values(by = 'date')
    prices_df.columns = [commodity]
    return prices_df
  
def get_predictors(commodity, predictors):
    ####################################################################
    # This returns the df containing the values of the commodity to be #
    # predicted as well as that of its predictors.                     #
    #                                                                  #
    # Args:                                                            #
    # 1) commodity: name of the commodity                              #
    # 2) predictors: the dataframe sent by Michael Button              #
    ####################################################################
    
    # get a list of all its predictors
    Pred = list(predictors.loc[predictors['INSTRUMENT'] == commodity]['INPUT'])
    Pred = [p for p in Pred if p != commodity]
    
    # Initialize the dataframe with the commodity we want to predict
    DF = from_csv( commodity )
    
    # Keep adding the predictors
    for predictor in Pred:
        temp_df = from_csv( predictor )
        DF = DF.join(temp_df)
        
    DF.index = pd.to_datetime(DF.index)
    return DF
  
def price_to_returns(prices_df, diff):
    ###############################################################
    # This changes the price data to 5 day log difference.        #
    # If X has negative values, use the following scheme:         # 
    #          X := X - min(X) + 1                                #
    ###############################################################
    
    # forward fill
    prices_df.fillna(method='ffill', inplace = True)
    
    returns_df = prices_df.copy()
    for colname in prices_df.columns:
        temp = prices_df.loc[:,colname]
        
        if np.min(temp) <= 0:
            temp = temp - np.min(temp) + 1
        returns_df[colname] = np.log(temp) - np.log(temp.shift(diff))

    # drop rows with NaN
    returns_df.dropna(inplace = True)
    return returns_df


  
def feature_generation(metal, time_pred = [1,5,22]):
  metal = metal + '_lme_prices'
  Metal_DF = get_predictors(metal, predictors)
  # remove comex, since comex is not 
  Metal_DF = Metal_DF[[col for col in list(Metal_DF.columns) if 'comex' not in col]]
  
  # 1, 5, 22 days
  # log difference L
  LD = [price_to_returns(Metal_DF, t_pred) for t_pred in time_pred]

  # EWMA of L
  # EWMA = [L.ewm(span = horizon).mean() for horizon, L in zip(time_pred, LD)]
  EWMA = [LD[0].ewm(halflife = horizon).mean() for horizon in time_pred ]

  # EWMV of L
  # 1. calculate expanding window mean for the returns (1 day) 
  # 2. subtract rolling mean and take square
  EM = LD[0].expanding(2).mean() 
  EWMV = [((LD[0] - EM)**2).ewm(span = horizon).mean()**0.5 for horizon in time_pred ]
  
  # rename columns
  for horizon, ld, ewma, ewmv in zip(time_pred, LD, EWMA, EWMV):
    ld.columns = [col + '_LD_' + str(horizon) for col in ld.columns]
    ewma.columns = [col + '_EWMA_' + str(horizon) for col in ewma.columns]
    ewmv.columns = [col + '_EWMV_' + str(horizon) for col in ewmv.columns]
    
  # merge together 
  ALL_FEATURES = pd.concat([pd.concat(DFS, axis = 1, sort=True) for DFS in [LD, EWMA, EWMV]], axis = 1)
  ALL_FEATURES_columns = list(ALL_FEATURES.columns)
  ALL_FEATURES_columns.sort()
  ALL_FEATURES = ALL_FEATURES.loc[:,ALL_FEATURES_columns]
  ALL_FEATURES.dropna(inplace = True)
  
  return ALL_FEATURES



def feature_extraction(DF, column_name, lag):
  # This extracts the relevant predictors for the column_name
  # threshold is the % of predictors that we want to include
  # Mutual Information
  DF_metal = DF.loc[:,[column_name]]
  MI_table = np.zeros(DF.shape[1])
  for i in range(DF.shape[1]):
      MI = sklearn.feature_selection.mutual_info_regression(DF_metal, DF.iloc[:,i])
      MI_table[i] = MI
  
  tol = 1e-3
  MI_table_2 = MI_table[MI_table > tol]
  out, bins = pd.qcut(MI_table_2, [0.9, 1], retbins=True)
  selected_cols = list(MI_table > bins[-2])
  
  DF = DF.loc[:,selected_cols].shift(periods = lag)
  DF.columns = [col + '_lag' for col in DF.columns]
  DF = pd.concat([DF_metal, DF], axis = 1, sort = True).dropna()
  time_column = pd.DataFrame(index = DF.index, 
                             data = np.arange(DF.shape[0])*0.01 , 
                             columns = ['time'] )
  DF = pd.concat([DF, time_column], axis = 1)
  # add time column
  
  
  return(DF)



# 1. Balanced Decision Standard Multitask GP

## 1.1 Some helper functions

In [0]:


class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, rank, kernel):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks= train_y.shape[0]
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            kernel, num_tasks= train_y.shape[0], rank=rank
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)




  
def MGP_train(X_train, Y_train, X_test, Y_test, kernel = gpytorch.kernels.RBFKernel, rank=2):
  likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks= Y_train.shape[0])
  model = MultitaskGPModel(X_train, Y_train, likelihood, rank = rank, 
                           kernel = kernel(ard_num_dims = X_train.shape[1]))

  model.train()
  likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam([
      {'params': model.parameters()},  # Includes GaussianLikelihood parameters
  ], lr=0.1)

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

  n_iter = 100
  for i in range(n_iter):
      optimizer.zero_grad()
      output = model(X_train)
      loss = -mll(output, Y_train)
      loss.backward()
      optimizer.step()
  model.eval()
  likelihood.eval()

  with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test))
    mean = predictions.mean

  mse = (mean - Y_test.t() ).numpy()**2
  direction = (torch.sign(mean) == torch.sign(Y_test.t())).numpy()

  return(np.mean(direction, axis = 0) , np.mean(mse, axis = 0))


class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(kernel)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
      

def GP_train(X_train, Y_train, X_test, Y_test, kernel = gpytorch.kernels.RBFKernel):
  likelihood = gpytorch.likelihoods.GaussianLikelihood()
  model = ExactGPModel(X_train, Y_train, likelihood, kernel(ard_num_dims = X_train.shape[1]))
  model.train()
  likelihood.train()

  # Use the adam optimizer
  optimizer = torch.optim.Adam([
      {'params': model.parameters()},  # Includes GaussianLikelihood parameters
  ], lr=0.1)

  # "Loss" for GPs - the marginal log likelihood
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
  training_iter = 100
  for l in range(training_iter):
      optimizer.zero_grad()
      output = model(X_train)
      loss = -mll(output, Y_train)
      loss.backward()
      optimizer.step()

  # Get into evaluation (predictive posterior) mode
  model.eval()
  likelihood.eval()

  # Make predictions by feeding model through likelihood
  with torch.no_grad(), gpytorch.settings.fast_pred_var():
      predictions = likelihood(model(X_test))
      mean = predictions.mean

  mse = (mean - Y_test.t() ).numpy()**2
  direction = (torch.sign(mean) == torch.sign(Y_test.t())).numpy()

  return(np.mean(direction) , np.mean(mse))

In [27]:
A = torch.randn(size = (5,2))
C = torch.randn(size = (5,5))
print(C.mm(torch.solve(A, C)[0]) -A)
C.mm(torch.inverse(C).mm(A)) - A



tensor([[ 1.2517e-06,  3.2783e-06],
        [-5.6624e-07, -4.3213e-07],
        [ 3.8743e-07, -1.3113e-06],
        [ 1.3709e-06, -5.9605e-07],
        [ 6.5565e-07,  1.6093e-06]])


tensor([[-4.3586e-06,  3.0398e-06],
        [-1.7285e-06,  2.8461e-06],
        [-2.5332e-06,  5.9605e-07],
        [ 3.4571e-06, -4.6492e-06],
        [ 8.3447e-07, -2.3246e-06]])

In [0]:
def kron(X, Y):
    Y_ = Y.repeat(X.shape[0],X.shape[1])
    y1,y2 = Y.shape
    x1,x2 = X.shape
    
    X_ = X.flatten()[:,None].repeat(1, y2).reshape(x1, x2*y2).repeat(1,y1).reshape(x1*y1, x2*y2)
    
    return(X_*Y_)
def softplus(param):
    return torch.log(1 + torch.exp(param) )

  
class Likelihood:
    def __init__(self):
        pass
    
    def forward(self, K, Y):
        Cholesky = torch.cholesky(K)
#         Likelihood = 0.5*2*torch.sum(torch.log(torch.diag(Cholesky))) \
#             + 0.5* (Y[None,:]).mm(torch.inverse(K) ).mm(Y[:,None]) 
        
        Likelihood = 0.5*2*torch.sum(torch.log(torch.diag(Cholesky))) \
            + 0.5* (Y[None,:]).mm(torch.solve(Y[:,None],K)[0] )
        return Likelihood
    
    
class RBF:
    def __init__(self, dimensions, ARD = False):
        self.sigma = torch.nn.Parameter(torch.randn(dimensions)) 
        self.ARD = ARD
        self.params = [self.sigma]
    
    def forward(self, X1, X2):
        
        if len(X1.shape) == 1:
            X1 = torch.unsqueeze(X1, 0)
            X2 = torch.unsqueeze(X2, 1)
            return torch.exp( -((X1 - X2)/softplus(self.sigma) )**2 )
        else:
            return torch.exp(-torch.norm((X1[:,None] - X2[None,:])/softplus(self.sigma), dim = 2)**2)

        
def predict(X, Y, x, Models, alphas = None):
    if alphas == None:
        alphas = torch.ones(len(Models), dtype = torch.float32)
    K_XX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_cov() for alpha,model in zip(alphas, Models)] )
    K_xX = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xX(x) for alpha,model in zip(alphas, Models)] )
    K_xx = reduce(lambda x, y: x + y, 
                  [alpha*model.get_K_xx(x) for alpha,model in zip(alphas, Models)] )
    pred_mean = K_xX.mm(torch.solve(Y[:,None],K_XX)[0] ).squeeze().reshape(-1,x.shape[0])
    
    #add noise
    Noise = reduce(lambda x, y: x + y, 
                  [alpha * softplus(model.noise) for alpha,model in zip(alphas, Models)] )
    
    
#     print(torch.mm(torch.mm(K_xX,torch.inverse(K_XX)), K_xX.t()))
    K_xX = K_xX.detach().numpy()
    K_XX = K_XX.detach().numpy()
    K_xx = K_xx.detach().numpy()
    T = K_xX @ np.linalg.solve(K_XX, K_xX.T) 
    
    pred_cov = K_xx - T + torch.diag(Noise).detach().numpy()
    
#     print(K_xX.mm(torch.inverse(K_XX)).mm(K_xX.t()))
    return(pred_mean, torch.tensor(pred_cov))
    
        
        
class MGP:
    def __init__(self, X, Y, kernel, rank, ARD):
        # X has size N x DIM
        # Y has size N X tasks
        self.X = X
        if len(self.X.shape) == 1:
            self.X = self.X[:,None]
        self.n_tasks = Y.shape[1]
        self.n_datapoints = Y.shape[0]
        
        self.Y = Y.t().flatten()
        self.kernel = kernel(X.shape[1], ARD)
        
        # initialize parameters
        self.noise = torch.nn.Parameter(torch.randn(self.n_tasks) ) 
        assert rank <= self.n_tasks
        # cov = task_cov task_cov^T
        self.task_cov = torch.nn.Parameter(torch.randn(self.n_tasks,rank) ) 
        
    def get_params(self):
        return self.kernel.params + [self.task_cov, self.noise]
    
    def get_cov(self):

        K = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(self.X, self.X))  \
            + kron(torch.diag(softplus(self.noise)), torch.eye(self.n_datapoints))
        return K
    
    def get_K_xX(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        K_xX = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(x, self.X))
        return K_xX
    
    def get_K_xx(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        K_xx = kron(self.task_cov.mm(self.task_cov.t()), self.kernel.forward(x, x))
        return K_xx
  
    

class SIGP: #simultaneous independent GP
    def __init__(self, X, Y, kernel, ARD):
        # X has size N x DIM
        # Y has size N X tasks
        self.X = X
        if len(self.X.shape) == 1:
            self.X = self.X[:,None]
        self.n_tasks = Y.shape[1]
        self.n_datapoints = Y.shape[0]
        self.Y = Y.t().flatten()
        self.kernels = [kernel(X.shape[1],ARD) for i in range(self.n_tasks)]
        
        # initialize parameters
        self.noise = torch.nn.Parameter(torch.randn(self.n_tasks) ) 

    def get_params(self):
        return reduce(lambda x, y: x+y, 
                      [ker.params for ker in self.kernels]) + [self.noise]
    
    def get_cov(self):
        
        list_kernels = [k.forward(self.X, self.X) for k in self.kernels]
        BD = Variable(torch.zeros(self.n_tasks * self.n_datapoints, 
                                  self.n_tasks * self.n_datapoints))
        for k, ker in enumerate(list_kernels):
            BD[ k * self.n_datapoints: (k+1) * self.n_datapoints,
                k * self.n_datapoints: (k+1) * self.n_datapoints] = ker
        
        K = BD + kron(torch.diag(softplus(self.noise)), torch.eye(self.n_datapoints))
        return K
    
    def get_K_xX(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        list_kernels = [k.forward(x, self.X) for k in self.kernels]
        K_xX = Variable(torch.zeros(self.n_tasks * x.shape[0] , self.n_tasks * self.n_datapoints))
        for k, ker in enumerate(list_kernels):
            K_xX[ k * x.shape[0]: (k+1) * x.shape[0],
                k * self.n_datapoints: (k+1) * self.n_datapoints] = ker
        return K_xX
    
    def get_K_xx(self, x):
        if len(x.shape) == 1:
            x = x[:,None]
        list_kernels = [k.forward(x, x) for k in self.kernels]
        K_xx = Variable(torch.zeros(self.n_tasks * x.shape[0] , self.n_tasks * x.shape[0]))
        for k, ker in enumerate(list_kernels):
            K_xx[ k * x.shape[0]: (k+1) * x.shape[0],
                  k * x.shape[0]: (k+1) * x.shape[0]] = ker
        return K_xx


## 1.2 CSMGP training functions

In [0]:
def CSMGP_train(X, Y, x, y, which_GP):
  if which_GP == 'MGP':
    model_MGP = MGP(X = X, Y= Y,kernel = RBF, rank = 2, ARD = False)
    models = [model_MGP]
  elif which_GP == 'SIGP':
    model_SIGP = SIGP(X = X, Y= Y,kernel = RBF, ARD = False)
    models = [model_SIGP]
  elif which_GP == 'all':
    model_MGP = MGP(X = X, Y= Y,kernel = RBF, rank = 2, ARD = False)
    model_SIGP = SIGP(X = X, Y= Y,kernel = RBF, ARD = False)
    models = [model_MGP, model_SIGP]
  
  LLH = Likelihood()
  alpha = torch.nn.Parameter(torch.tensor(0.))
  optimizer = torch.optim.Adam(params = reduce(lambda x,y: x+y, [model.get_params() for model in models]) + [alpha], 
                               lr = 0.1)
  llh = []

  for i in range(200):
      if which_GP == 'MGP':
        _alpha = torch.tensor(1., dtype = torch.float32)
        K = model_MGP.get_cov()
      elif which_GP == 'SIGP':
        _alpha = torch.tensor(1., dtype = torch.float32)
        K = model_SIGP.get_cov()
      elif which_GP == 'all':      
        _alpha = torch.sigmoid(alpha)
        K = _alpha*model_SIGP.get_cov() \
            + (1-_alpha)*model_MGP.get_cov()
      
      loss = LLH.forward(K, models[0].Y)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if i%10 == 0:
#           print(loss.detach().numpy().squeeze())
          llh.append(loss.detach().numpy().squeeze())
  
  alphas = [_alpha, 1- _alpha] * (which_GP == 'all') \
          + [torch.tensor(1., dtype = torch.float32)] * (which_GP != 'all')
  pred_mean, pred_cov = predict(X = X, 
                                Y = models[0].Y,
                                Models = models, 
                                alphas = alphas, 
                                x=x) 
  
  
  
  CSMGP_sign = np.mean((torch.sign(pred_mean.t()) == torch.sign(y)).detach().numpy(), axis = 0)
  CSMGP_error = np.mean((pred_mean.t() - y).detach().numpy()**2, axis = 0)
#   print(_alpha)
  return(CSMGP_sign, CSMGP_error, llh)
  

# 2. Toy Dataset

In [0]:
trials = 200

SIGP_sign = np.zeros((2,trials))
SIGP_mse = np.zeros((2,trials))
MGP_sign = np.zeros((2,trials))
MGP_mse = np.zeros((2,trials))
_all_sign = np.zeros((2,trials))
_all_mse  = np.zeros((2,trials))


for i in range(trials):
  X = torch.cat([14*torch.rand(15)[:,None], 14*torch.rand(15)[:,None]], dim = 1) 
  Y = torch.cat([torch.sin(X)[:,[0]], 
                 torch.cos(2*X)[:,[1]]], dim=1) 

#   x = torch.cat([torch.linspace(0,14,10)[:,None], torch.linspace(-0,7,10)[:,None]], dim = 1)
  x = torch.cat([14*torch.rand(10)[:,None], 14*torch.rand(10)[:,None]], dim = 1) 
  y = torch.cat([torch.sin(x[:,[0]]), torch.cos(2*x[:,[1]])], dim = 1)
  
  noise = torch.randn(size = Y.shape) 
  
  Y = Y + torch.cat([0.1*noise[:,[0]], 0.25*noise[:,[1]] ], dim = 1)
  
  SIGP_sign[:,i], SIGP_mse[:,i], llh = CSMGP_train(X, Y, x, y, which_GP = 'SIGP')
  MGP_sign[:,i], MGP_mse[:,i], llh = CSMGP_train(X, Y, x, y, which_GP = 'MGP')
#   for j in range(2):
#     SIGP_sign[j,i], SIGP_mse[j,i] = GP_train(X, Y[:,j].t().contiguous(), x, y[:,j])
# #     print(GP_train(X, Y[:,j].t().contiguous(), x, y[:,j]))
#   MGP_sign[:,i], MGP_mse[:,i] = MGP_train(X, Y.t().contiguous(), x, y.t().contiguous())
  _all_sign[:,i], _all_mse[:,i], llh= CSMGP_train(X, Y, x, y, which_GP = 'all')

[tensor(1.)]
tensor([[6.1463e-01, 7.8703e-01, 2.0413e-04, 1.7327e-01, 8.9564e-02, 3.7981e-01,
         1.6295e-01, 2.1692e-05, 2.8572e-06, 9.2259e-01, 6.6976e-03, 1.8817e-09,
         4.2736e-03, 7.9584e-01, 1.6756e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [7.3929e-01, 7.6505e-01, 1.9108e-04, 2.6638e-01, 9.7081e-02, 3.2553e-01,
         1.9145e-01, 1.7237e-05, 1.3809e-06, 8.2826e-01, 5.2472e-03, 1.5057e-09,
         3.7419e-03, 6.5524e-01, 1.5972e-04, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.1806e-03, 1.2210e-01, 3.3130e-01, 6.9562e-05, 7.7047e-01, 2.7457e-04,
         5.7171e-01, 1.4179e-01, 5.0109e-02, 9.8836e-03, 8.7994e-01, 1.1408e-03,
         8.00

KeyboardInterrupt: ignored

In [0]:
print('SIGP')
print(np.std(SIGP_mse, axis = 1))
print(np.mean(SIGP_mse, axis = 1))
print('MGP')
print(np.std(MGP_mse, axis = 1))
print(np.mean(MGP_mse, axis = 1))
print('CSMGP')
print(np.std(_all_mse, axis = 1))
print(np.mean(_all_mse, axis = 1))

np.mean(_all_mse,axis = 1)/np.mean(SIGP_mse,axis = 1)

SIGP
[0.18434206 0.24364233]
[0.15109948 0.43234603]
MGP
[0.22959163 0.20210867]
[0.27491605 0.48586407]
CSMGP
[0.11661971 0.18169688]
[0.12523025 0.37860405]


array([0.82879342, 0.87569684])

In [0]:
print('SIGP')
print(np.mean(SIGP_sign, axis = 1))
print(np.mean(SIGP_mse, axis = 1))
print('MGP')
print(np.mean(MGP_sign, axis = 1))
print(np.mean(MGP_mse, axis = 1))
print('CSMGP')
print(np.mean(_all_sign, axis = 1))
print(np.mean(_all_mse, axis = 1))

np.mean(_all_mse,axis = 1)/np.mean(SIGP_mse,axis = 1)

SIGP
[0.854 0.625]
[0.14512087 0.39264165]
MGP
[0.7495 0.5435]
[0.26089253 0.47250498]
CSMGP
[0.887  0.6695]
[0.13290948 0.3762909 ]


array([0.91585369, 0.95835706])

# 3. Commodities Dataset

In [0]:
def Across_Metals(window, horizon, Target_Variables, trials):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  # this
  SIGP_sign = np.zeros((I,trials))
  SIGP_mse = np.zeros((I,trials))
  MGP_sign = np.zeros((I,trials))
  MGP_mse = np.zeros((I,trials))
  _all_sign = np.zeros((I,trials))
  _all_mse  = np.zeros((I,trials))

  BM_mse = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = DF.drop(Target_Variables, axis = 1)
  
  for i in range(trials):
    print(i)
    START = int(np.random.choice(np.arange(len(Y) - 200),1) )
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.squeeze() , dtype = torch.float32)
    
#     SIGP_sign[:,i], SIGP_mse[:,i] , llh= CSMGP_train(X_train, Y_train, X_test, Y_test, which_GP = 'SIGP')
#     MGP_sign[:,i], MGP_mse[:,i] , llh1= CSMGP_train(X_train, Y_train, X_test, Y_test, which_GP = 'MGP')
    for j in range(2):
      SIGP_sign[j,i], SIGP_mse[j,i] = GP_train(X_train, Y_train[:,j].t().contiguous(), X_test, Y_test[j])
    MGP_sign[:,i], MGP_mse[:,i] = MGP_train(X_train, Y_train.t().contiguous(), X_test, Y_test)
    _all_sign[:,i], _all_mse[:,i], llh= CSMGP_train(X_train, Y_train, X_test, Y_test, which_GP = 'all')
    
    BM_mse[:,i] = Y_test**2
    
      
  return SIGP_sign, SIGP_mse, MGP_sign, MGP_mse, _all_sign, _all_mse, BM_mse, llh




In [0]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [1, 5, 22]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  SIGP_sign, SIGP_mse, MGP_sign, MGP_mse, _all_sign, _all_mse, BM_mse, llh = Across_Metals(window = 100, 
                                                                                           horizon = horizon, 
                                                                                           Target_Variables = Target_Variables, 
                                                                                           trials = 200)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    print('CSMGP MSE', np.mean(_all_mse[tv,:] ) )
    print('CSMGP sign', np.mean(_all_sign[tv,:]) )
    print(' ')
    
    print('MGP MSE', np.mean(MGP_mse[tv,:] ))
    print('MGP sign', np.mean(MGP_sign[tv,:] ))
    print(' ')
    
    print('SIGP MSE', np.mean(SIGP_mse[tv,:] ))
    print('SIGP sign', np.mean(SIGP_sign[tv,:] ))      
    print(' ')    
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
T+ 1
AL :
CSMGP MSE 0.0002525979452742166
CSMGP sign 0.475
 
MGP MSE 0.0002212709520340228
MGP sign 0.42
 
SIGP MSE 0.525
SIGP sign 0.00021520800798166455
 
Benchmark 0.00021811353795992973
 
CU :
CSMGP MSE 0.000375747378278154
CSMGP sign 0.51
 
MGP MSE 0.00038655081860920147
MGP sign 0.5
 
SIGP MSE 0.445
SIG

In [0]:

#Statistical Significance
Joint_Predictions = [ ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [22]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  SIGP_sign, SIGP_mse, MGP_sign, MGP_mse, _all_sign, _all_mse, BM_mse, llh = Across_Metals(window = 100, 
                                                                                           horizon = horizon, 
                                                                                           Target_Variables = Target_Variables, 
                                                                                           trials = 1000)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    print('CSMGP MSE', np.mean(_all_mse[tv,:] ) )
    print('CSMGP sign', np.mean(_all_sign[tv,:]) )
    print(' ')
    
    print('MGP MSE', np.mean(MGP_mse[tv,:] ))
    print('MGP sign', np.mean(MGP_sign[tv,:] ))
    print(' ')
    
    print('SIGP MSE', np.mean(SIGP_mse[tv,:] ))
    print('SIGP sign', np.mean(SIGP_sign[tv,:] ))      
    print(' ')    
          
    print('Benchmark', np.mean(BM_mse[tv,:] ) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
def Across_Metals_CI(window, horizon, Target_Variables, trials):

  # metal is either 'al' or 'cu'
  # window is the length of data before prediction
  # Horizons is a list of numbers (e.g. [1,5,22])
  # Target_Variables is a list of strings 
  #      (e.g. ['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'])
  # kernel is the kernel that we will use 
  # trials (self explanatory)
  # n_test (number of test points per trial)
  
  I = len(Target_Variables)
  
  # this
  _all_Included = np.zeros((I,trials))
  _all_Interval  = np.zeros((I,trials))

  BM_mse = np.zeros((I,trials))
  
  # this contains the list of dataframes 
  DF_list = []
  for TV_index,TV in enumerate(Target_Variables):
    # horizon and target variable
    DF = feature_generation(TV[0:2], time_pred = [1,5,22])
    DF = feature_extraction(DF, TV, horizon)
    DF_list.append(DF)
  
  # now join the DFs in DF_list, and remove duplicate columns
  DF = pd.concat(DF_list, axis = 1, sort = True).dropna()
  DF = DF.loc[:,~DF.columns.duplicated()]
  Y = DF.loc[:,Target_Variables]
  X = DF.drop(Target_Variables, axis = 1)
  
  Chosen = np.random.choice(np.arange(len(Y) - 200),trials, replace= False) 
  
  for i in range(trials):
    print(i)
    START = Chosen[i]
    END = START + window
    X_train = torch.tensor( X.iloc[START:END, :].values , dtype = torch.float32)
    X_test = torch.tensor( X.iloc[[(END+horizon-1)], :].values , dtype = torch.float32)
    Y_train = torch.tensor( Y.iloc[START:END, :].values.squeeze() , dtype = torch.float32)
    Y_test = torch.tensor( Y.iloc[[(END+horizon-1)], :].values.squeeze() , dtype = torch.float32)
    
    pred_mean, pred_cov = CSMGP_train_CI(X_train, Y_train, X_test, Y_test, which_GP = 'all')    
    

    
    upper = (pred_mean.squeeze() + 1.96*pred_cov).detach().numpy()
    lower = (pred_mean.squeeze() - 1.96*pred_cov).detach().numpy()
#     print((pred_mean.squeeze() + 2*pred_cov > Y_test).numpy() )
#     print((pred_mean.squeeze() - 2*pred_cov < Y_test).numpy() )
    _all_Included[:,i] = (upper > Y_test.numpy()) * (lower < Y_test.numpy())
    _all_Interval[:,i] = 2*1.96 * pred_cov.detach().numpy()
    
    
    
  return _all_Included, _all_Interval


def CSMGP_train_CI(X, Y, x, y, which_GP):
  if which_GP == 'MGP':
    model_MGP = MGP(X = X, Y= Y,kernel = RBF, rank = 2, ARD = False)
    models = [model_MGP]
  elif which_GP == 'SIGP':
    model_SIGP = SIGP(X = X, Y= Y,kernel = RBF, ARD = False)
    models = [model_SIGP]
  elif which_GP == 'all':
    model_MGP = MGP(X = X, Y= Y,kernel = RBF, rank = 2, ARD = False)
    model_SIGP = SIGP(X = X, Y= Y,kernel = RBF, ARD = False)
    models = [model_MGP, model_SIGP]
  
  LLH = Likelihood()
  alpha = torch.nn.Parameter(torch.tensor(0.))
  optimizer = torch.optim.Adam(params = reduce(lambda x,y: x+y, [model.get_params() for model in models]) + [alpha], 
                               lr = 0.1)
  llh = []

  for i in range(200):
      if which_GP == 'MGP':
        _alpha = torch.tensor(1., dtype = torch.float32)
        K = model_MGP.get_cov()
      elif which_GP == 'SIGP':
        _alpha = torch.tensor(1., dtype = torch.float32)
        K = model_SIGP.get_cov()
      elif which_GP == 'all':      
        _alpha = torch.sigmoid(alpha)
        K = _alpha * model_MGP.get_cov() \
            + (1-_alpha)*model_SIGP.get_cov()
      
      loss = LLH.forward(K, models[0].Y) - 2*torch.log(_alpha * (1-_alpha)) 
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      if i%10 == 0:
#           print(loss.detach().numpy().squeeze())
          llh.append(loss.detach().numpy().squeeze())
  
  alphas = [_alpha, 1- _alpha] * (which_GP == 'all') \
          + [torch.tensor(1., dtype = torch.float32)] * (which_GP != 'all')
  
  pred_mean, pred_cov = predict(X = X, 
                                Y = models[0].Y,
                                Models = models, 
                                alphas = alphas, 
                                x=x) 
  pred_cov = torch.diag(pred_cov)**0.5
  
  return(pred_mean, pred_cov)
  

In [44]:
Joint_Predictions = [['al_lme_prices_LD_1', 'cu_lme_prices_LD_1'],
                     ['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [1, 5, 22]

Joint_Predictions = [['al_lme_prices_LD_5', 'cu_lme_prices_LD_5'],
                     ['al_lme_prices_LD_22', 'cu_lme_prices_LD_22'] ]

horizons = [5, 22]

for h, horizon in enumerate(horizons):
  Target_Variables = Joint_Predictions[h]
  _all_Included, _all_Interval = Across_Metals_CI(window = 100, 
                                                   horizon = horizon, 
                                                   Target_Variables = Target_Variables, 
                                                   trials = 1)
  print('T+',horizon)
  for tv, TV in enumerate(Target_Variables):
    print(TV[0:2].upper(),':') #this is the metal
    
    print('CSMGP Included', np.mean(_all_Included[tv,:] ) )
    print('CSMGP Interval', np.mean(_all_Interval[tv,:]) )
    print(' ')
    
  print('######')

0
1
2
3
4
5
6
7
8
9
T+ 5
AL :
CSMGP Included 0.9
CSMGP Interval 0.13740107417106628
 
CU :
CSMGP Included 0.7
CSMGP Interval 0.16218212693929673
 
######
0
1
2
3
4
5
6
7
8
9
T+ 22
AL :
CSMGP Included 0.5
CSMGP Interval 0.4686974361538887
 
CU :
CSMGP Included 0.7
CSMGP Interval 0.5600805386900902
 
######


In [49]:
A = np.random.normal(size = (5,2))
C = np.random.normal(size = (5,5))
C = C @ C.T

A.T @ C@ A

A[:,[0]].T @ C @ A[:,[0]]

array([[19.8422252]])

In [50]:
A.T @ C@ A


array([[19.8422252 , 16.73735129],
       [16.73735129, 16.86399004]])